<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


## Final Project - Build an ML Pipeline for Airfoil noise prediction


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at an aeronautics consulting company. Your company prides itself in being able to efficiently design airfoils for use in planes and sports cars. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able to do ETL jobs and build ML pipelines. In this project you will use the modified version of the NASA Airfoil Self Noise dataset. You will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. You will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. You will evaluate the model and towards the end you will persist the model.



## Objectives

In this 4 part assignment you will:

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model 
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise
 
 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


## Before you Start


Before you start attempting this project it is highly recommended that you finish the practice project.


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

### Task 2 - Create a spark session


In [5]:
#Create a SparkSession

spark = SparkSession.builder \
    .appName("CHILD_LABOR") \
    .getOrCreate()

### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2023-11-24 13:00:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.002s  

2023-11-24 13:00:36 (27.1 MB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



Load the dataset into the spark dataframe


In [8]:
# Load the dataset that you have downloaded in the previous task

df = df = spark.read.csv("CHILD.CSV", header=True, inferSchema=True)


### Task 4 - Print top 5 rows of the dataset


In [9]:
#your code goes here
df.show(5)

+--------------------+---------------------------+-----------------------------+---------------------------------------+-----------------------------+--------------------------+--------------------+------------------------+
|             Poverty|Lack_of_access_to_education|Cultural_norms_and_traditions|Lack_of_enforcement_of_child_labor_laws|Global_demand_for_cheap_labor|Urbanization_and_migration|Family_circumstances|Conflict_and_emergencies|
+--------------------+---------------------------+-----------------------------+---------------------------------------+-----------------------------+--------------------------+--------------------+------------------------+
|Families living i...|       Inadequate or una...|         In some cultures,...|                   Weak enforcement ...|         Industries seekin...|      Rural families mi...|Orphaned, abandon...|    During times of c...|
+--------------------+---------------------------+-----------------------------+------------------------

### Task 6 - Print the total number of rows in the dataset


In [9]:
rowcount1 = df.count()
print(rowcount1)

1522


### Task 7 - Drop all the duplicate rows from the dataset


In [10]:
df = df.dropDuplicates()

### Task 8 - Print the total number of rows in the dataset


In [11]:
#your code goes here

rowcount2 = df.count()
print(rowcount2)


[Stage 6:=====================================================> (195 + 5) / 200]

1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [12]:
df = df.na.drop()

### Task 10 - Print the total number of rows in the dataset


In [13]:
#your code goes here

rowcount3 = df.count()
print(rowcount3)


[Stage 9:======================================================>(198 + 2) / 200]

1499


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"Drop


In [14]:
df = df.withColumnRenamed("SoundLevel", "SoundLevelDecibels").drop("SoundLevel")

### Task 12 - Save the dataframe in parquet formant, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [10]:
df.write.parquet("CHILD.parquet")

#### Part 1 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [12]:
df.printSchema()

root
 |-- Poverty: string (nullable = true)
 |-- Lack_of_access_to_education: string (nullable = true)
 |-- Cultural_norms_and_traditions: string (nullable = true)
 |-- Lack_of_enforcement_of_child_labor_laws: string (nullable = true)
 |-- Global_demand_for_cheap_labor: string (nullable = true)
 |-- Urbanization_and_migration: string (nullable = true)
 |-- Family_circumstances: string (nullable = true)
 |-- Conflict_and_emergencies: string (nullable = true)



## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [11]:
df = spark.read.parquet("CHILD.parquet")

### Task 2 - Print the total number of rows in the dataset


In [19]:
#your code goes here

rowcount4 = df.count()
print(rowcount4)



[Stage 14:===========================================>              (6 + 2) / 8]

1499


### Task 3 - Define the VectorAssembler pipeline stage


Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [14]:
from pyspark.ml.feature import StringIndexer

# Assuming df is your DataFrame
# Iterate through columns and apply StringIndexer to categorical columns
for col in df.columns:
    if col != "Lack_of_access_to_education":  # Exclude the target column if it's categorical
        indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
        df = indexer.fit(df).transform(df)

### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [15]:
from pyspark.ml.feature import StandardScaler

# Assuming your inputCol is "features" and outputCol is "scaledFeatures"
scaler = StandardScaler(inputCol="_index", outputCol="scaledissue", withStd=True, withMean=True)


### Task 5 - Define the StandardScaler pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"

**Note:You need to use the scaledfeatures retreived in the previous step.**


In [16]:
lr = LinearRegression(featuresCol="_index", labelCol="scaledissue")

### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [17]:
stages = [assembler, scaler, lr]

# Create a pipeline
pipeline = Pipeline(stages=stages)

# Fit the pipeline model
pipeline_model = pipeline.fit(df)

IllegalArgumentException: 'Data type string of column Poverty is not supported.\nData type string of column Cultural_norms_and_traditions is not supported.\nData type string of column Lack_of_enforcement_of_child_labor_laws is not supported.\nData type string of column Global_demand_for_cheap_labor is not supported.\nData type string of column Urbanization_and_migration is not supported.\nData type string of column Family_circumstances is not supported.\nData type string of column Conflict_and_emergencies is not supported.'

### Task 7 - Split the data


In [26]:
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

### Task 8 - Fit the pipeline


In [39]:
# Fit the pipeline using the training data
# your code goes here

pipeline_model = pipeline.fit(trainingData)

23/11/24 13:37:53 WARN util.Instrumentation: [9ae3c1f6] regParam is zero, which might cause numerical instability and overfitting.


#### Part 2 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [28]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [30]:
predictions = pipeline_model.transform(testingData)

# Show some of the predictions
predictions.select("features", "SoundLevelDecibels", "prediction").show(5)

[Stage 54:>                                                         (0 + 1) / 1]

+--------------------+------------------+------------------+
|            features|SoundLevelDecibels|        prediction|
+--------------------+------------------+------------------+
|[200.0,9.9,0.1524...|           127.315|123.64344009624753|
|[200.0,15.4,0.050...|           119.975|123.48695788614877|
|[200.0,15.4,0.050...|           121.783|124.38983849684254|
|[250.0,19.7,0.050...|           127.224|121.44706993294302|
|[315.0,4.0,0.2286...|           122.229|125.68312652454188|
+--------------------+------------------+------------------+
only showing top 5 rows



### Task 2 - Print the MSE


In [32]:
from pyspark.ml.evaluation import RegressionEvaluator

# Assuming predictions is your DataFrame with prediction and actual values
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mse")

# Calculate MSE
mse = evaluator.evaluate(predictions)
print("Mean Squared Error (MSE):", mse)

[Stage 55:>                                                         (0 + 8) / 8]

Mean Squared Error (MSE): 22.593754071348812


### Task 3 - Print the MAE


In [33]:
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mae")

# Calculate MAE
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error (MAE):", mae)

[Stage 57:====================================>                     (5 + 3) / 8]

Mean Absolute Error (MAE): 3.7336902294631287


### Task 4 - Print the R-Squared(R2)


In [34]:
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="r2")

# Calculate R-squared (R2)
r2 = evaluator.evaluate(predictions)
print("R-squared (R2):", r2)

[Stage 59:==============>                                           (2 + 6) / 8]

R-squared (R2): 0.5426016508689058


#### Part 3 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [41]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipeline_model.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  22.59
Mean Absolute Error =  3.73
R Squared =  0.54
Intercept =  132.6


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [42]:
pipeline_model.write().overwrite().save("Final_Project")

### Task 2 - Load the model from the path "Final_Project"


In [43]:
# Load the pipeline model you have created in the previous step
loaded_pipeline_model = PipelineModel.load("Final_Project")


### Task 3 - Make predictions using the loaded model on the testdata


In [50]:
predictions = loaded_pipeline_model.transform(testingData).show(5)

[Stage 91:>                                                         (0 + 1) / 1]

+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|            features|      scaledFeatures|        prediction|
+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|      200|          9.9|     0.1524|              39.6|              0.0233328|           127.315|[200.0,9.9,0.1524...|[0.06338939572741...|123.64344009624753|
|      200|         15.4|     0.0508|              31.7|              0.0289853|           119.975|[200.0,15.4,0.050...|[0.06338939572741...|123.48695788614877|
|      200|         15.4|     0.0508|              39.6|              0.0282593|           121.783|[200.0,15.4,0.050...|[0.06338939572741...|124.38983849684254|
|      250|         19.7|     0.05

### Task 4 - Show the predictions


In [51]:
predictions = loaded_pipeline_model.transform(testingData).show(5)

[Stage 92:>                                                         (0 + 1) / 1]

+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|            features|      scaledFeatures|        prediction|
+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|      200|          9.9|     0.1524|              39.6|              0.0233328|           127.315|[200.0,9.9,0.1524...|[0.06338939572741...|123.64344009624753|
|      200|         15.4|     0.0508|              31.7|              0.0289853|           119.975|[200.0,15.4,0.050...|[0.06338939572741...|123.48695788614877|
|      200|         15.4|     0.0508|              39.6|              0.0282593|           121.783|[200.0,15.4,0.050...|[0.06338939572741...|124.38983849684254|
|      250|         19.7|     0.05

#### Part 4 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [55]:
print("Part 4 - Evaluation")

loadedmodel = loaded_pipeline_model.stages[-1]
totalstages = len(loaded_pipeline_model.stages)
inputcolumns = loaded_pipeline_model.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.9728
Coefficient for AngleOfAttack is -2.4775
Coefficient for ChordLength is -3.3818
Coefficient for FreeStreamVelocity is 1.5789
Coefficient for SuctionSideDisplacement is -1.6465


### Stop Spark Session


In [56]:
spark.stop()

## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
